# Forest Cover Type Prediction
W207 Spring 2020

Elevation - Elevation in meters\
Aspect - Aspect in degrees azimuth (aspect denotes the compass direction in which the slope of a mountain faces)\
Slope - Slope in degrees (Slope is the steepness or the degree of incline of a surface)\
Horizontal_Distance_To_Hydrology - Horz Dist to nearest surface water features\
Vertical_Distance_To_Hydrology - Vert Dist to nearest surface water features\
Horizontal_Distance_To_Roadways - Horz Dist to nearest roadway\
Hillshade_9am (0 to 255 index) - Hillshade index at 9am, summer solstice\
Hillshade_Noon (0 to 255 index) - Hillshade index at noon, summer solstice\
Hillshade_3pm (0 to 255 index) - Hillshade index at 3pm, summer solstice\
Horizontal_Distance_To_Fire_Points - Horz Dist to nearest wildfire ignition points\
Wilderness_Area (4 binary columns, 0 = absence or 1 = presence) - Wilderness area designation\
Soil_Type (40 binary columns, 0 = absence or 1 = presence) - Soil Type designation\
Cover_Type (7 types, integers 1 to 7) - Forest Cover Type designation

## Import Libraries & Data

In [46]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

# SK-learn libraries for learning.
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from scipy import stats

In [4]:
# Data source: https://www.kaggle.com/c/forest-cover-type-prediction/overview
# Load data

train_path = '../data/train.csv'
# unlabeled_path = '../data/test.csv'
train_csv = np.genfromtxt(train_path, delimiter=',', names=True)
unlabeled_csv = np.genfromtxt(unlabeled_path, delimiter=',', names=True)

# Extract feature and label strings
all_headers = train_csv.dtype.names
feature_name = unlabeled_csv.dtype.names
label_name = set(train_csv['Cover_Type'])

## Summary Statistics

In [6]:
df = pd.DataFrame(train_csv, columns=[i for i in all_headers])
stat_summary = df.describe().T
stat_summary

,count,mean,std,min,25%,50%,75%,max
Id,15120.0,7560.500000,4364.912370,1.0,3780.75,7560.5,11340.25,15120.0
Elevation,15120.0,2749.322553,417.678187,1863.0,2376.00,2752.0,3104.00,3849.0
Aspect,15120.0,156.676653,110.085801,0.0,65.00,126.0,261.00,360.0
Slope,15120.0,16.501587,8.453927,0.0,10.00,15.0,22.00,52.0
Horizontal_Distance_To_Hydrology,15120.0,227.195701,210.075296,0.0,67.00,180.0,330.00,1343.0
Vertical_Distance_To_Hydrology,15120.0,51.076521,61.239406,-146.0,5.00,32.0,79.00,554.0
Horizontal_Distance_To_Roadways,15120.0,1714.023214,1325.066358,0.0,764.00,1316.0,2270.00,6890.0
Hillshade_9am,15120.0,212.704299,30.561287,0.0,196.00,220.0,235.00,254.0
Hillshade_Noon,15120.0,218.965608,22.801966,99.0,207.00,223.0,235.00,254.0
Hillshade_3pm,15120.0,135.091997,45.895189,0.0,106.00,138.0,167.00,248.0


In [7]:
# Exclude columns with no useful data for prediction
new_headers = [n for n in all_headers if n not in ['Id', 'Soil_Type7', 'Soil_Type15']]
df = df[new_headers]

In [12]:
# Distribution of cover types
df.groupby('Cover_Type').size()

Cover_Type
1.0    2160
2.0    2160
3.0    2160
4.0    2160
5.0    2160
6.0    2160
7.0    2160
dtype: int64

In [33]:
# Check correlation between features

df_corr = df.drop(['Cover_Type'], axis=1)
corrMatrix = df_corr.corr()
sn.heatmap(corrMatrix, annot=True)

In [41]:
# Top ten correlations
c = corrMatrix.unstack().abs()
c[c!=1].sort_values(ascending = False)[:10]

Wilderness_Area4                  Elevation                           0.783651
Elevation                         Wilderness_Area4                    0.783651
Hillshade_9am                     Hillshade_3pm                       0.779965
Hillshade_3pm                     Hillshade_9am                       0.779965
Horizontal_Distance_To_Hydrology  Vertical_Distance_To_Hydrology      0.652142
Vertical_Distance_To_Hydrology    Horizontal_Distance_To_Hydrology    0.652142
Hillshade_3pm                     Aspect                              0.635022
Aspect                            Hillshade_3pm                       0.635022
Hillshade_3pm                     Hillshade_Noon                      0.614526
Hillshade_Noon                    Hillshade_3pm                       0.614526
dtype: float64

**Learning:**
- All features have 15120 observations so no missing data. All features could be used.
- 'Id', 'Soil_Type7', and 'Soil_Type15' has no meaningful data based on summary statistics. Will drop from datasets
- Negative values for Vertical_Distance_To_Hydrology
- One hot encoding (a single high bit and all the others low) for Wilderness Area and Soil Type
- Balanced distribution of cover types (2160 each)
- There are some correlations among features, possibly potential to reduce feature set through transformation or PCA

## Rescaling and Standardizing

In [85]:
# Function to rescale bounded, continuous features: All continuous input that are bounded, rescale them to [-1, 1].  x = (2x - max - min)/(max - min)
def rescale(feature, minimum, maximum):
    return (feature * 2 - minimum - maximum) / (maximum - minimum)

# Standardize all continuous features: x = (x - mean)/stdev
def standardize(feature, mean, stdev):
    return (feature - mean) / stdev

bounded_features = ['Aspect', 'Slope', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']
bounded_min_max = [(0, 360), (0,90), (0, 255), (0, 255), (0, 255)]

continuous_features = ['Elevation', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 
                       'Horizontal_Distance_To_Roadways', 'Horizontal_Distance_To_Fire_Points']
continuous_mean_std = [(stat_summary.loc[i, 'mean'], (stat_summary.loc[i, 'std'])) for i in continuous_features]

# Create new columns for rescaled features
for i, j in enumerate(bounded_features):
    # Create new rescaled column for bounded features
    df[j.lower() + "_RS"] = rescale(df[j], bounded_min_max[i][0], bounded_min_max[i][1])

# Create new columns for standardized features    
for m, n in enumerate(continuous_features):
    # Create new standardize column for continuous features
    df[n.lower() + "_RS"] = standardize(df[n], continuous_mean_std[m][0], continuous_mean_std[m][1])

excluded_columns = ['Id', 'Soil_Type7', 'Soil_Type15'] + bounded_features + continuous_features

df = df[[c for c in df.columns if c not in excluded_columns]]

# Split df: labels, binary, and non-binary
labels = df.pop("Cover_Type").to_numpy()
non_binary_features = [s for s in df.columns if "_RS" in s]
binary_features = [[s for s in df.columns if "_RS" not in s]]

## Split 70/20/10

In [88]:
# Train, dev, test split (70/20/10)
split1 = int(len(df)* 0.70)
split2 = int(split1 + (len(train_csv) - split1) * 2 / 3)

train_data, train_labels = df[:split1].to_numpy(), labels[:split1]
dev_data, dev_labels     = df[split1:split2].to_numpy(), labels[split1:split2]
test_data, test_labels   = df[split2:], labels[split2:]

print('training label shape:', train_labels.shape)
print('dev label shape:',      dev_labels.shape)
print('test label shape:',     test_labels.shape)
print('labels names:',         label_name)
print('number of features:',   len(train_data[1]))
print('binary feature names:\n',   binary_features)
print('non binary feature names:\n',   non_binary_features)

training label shape: (10584,)
dev label shape: (3024,)
test label shape: (1512,)
labels names: {1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0}
number of features: 52
binary feature names:
 [['Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40']]
non binary feature names:
 ['aspect_RS', 'slope_RS', 'hillshade_9am_RS', 'hillshade_noon_RS', 'hillshade_3pm_RS', 'elevation_RS', 'horizontal_distance_to_hydrology_RS', 'vertica

## Feature Selection (Random Forest Classifier)

In [100]:
# Random Forest Classifier
sel = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1))
sel.fit(train_data, train_labels)

# Feature importance cut off weight
print(np.mean(sel.estimator_.feature_importances_))

# Show which features selected as "important"
selected_features_RF = df.columns[sel.get_support()]
selected_features_RF

0.019230769230769225


Index(['Wilderness_Area4', 'Soil_Type3', 'aspect_RS', 'slope_RS',
       'hillshade_9am_RS', 'hillshade_noon_RS', 'hillshade_3pm_RS',
       'elevation_RS', 'horizontal_distance_to_hydrology_RS',
       'vertical_distance_to_hydrology_RS',
       'horizontal_distance_to_roadways_RS',
       'horizontal_distance_to_fire_points_RS'],
      dtype='object')

In [101]:
# Resplit based on RF selected features

df_RF = df[selected_features_RF]

train_data_RF, train_labels_RF = df_RF[:split1].to_numpy(), labels[:split1]
dev_data_RF, dev_labels_RF     = df_RF[split1:split2].to_numpy(), labels[split1:split2]
test_data_RF, test_labels_RF   = df_RF[split2:], labels[split2:]

## Feature Selection (Recursive Feature Elimination)

In [113]:
from sklearn.feature_selection import RFE
sel = RFE(RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1), n_features_to_select=30)
sel.fit(train_data, train_labels)

# Feature importance cut off weight
print(np.mean(sel.estimator_.feature_importances_))

# Show which features selected as "important"
selected_features_RFE = df.columns[sel.get_support()]
selected_features_RFE

0.03333333333333333


Index(['Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3',
       'Wilderness_Area4', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4',
       'Soil_Type10', 'Soil_Type12', 'Soil_Type13', 'Soil_Type17',
       'Soil_Type22', 'Soil_Type23', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type32', 'Soil_Type33', 'Soil_Type38', 'Soil_Type39',
       'Soil_Type40', 'aspect_RS', 'slope_RS', 'hillshade_9am_RS',
       'hillshade_noon_RS', 'hillshade_3pm_RS', 'elevation_RS',
       'horizontal_distance_to_hydrology_RS',
       'vertical_distance_to_hydrology_RS',
       'horizontal_distance_to_roadways_RS',
       'horizontal_distance_to_fire_points_RS'],
      dtype='object')

In [114]:
# Resplit based on RF selected features

df_RFE = df[selected_features_RFE]

train_data_RFE, train_labels_RFE = df_RFE[:split1].to_numpy(), labels[:split1]
dev_data_RFE, dev_labels_RFE     = df_RFE[split1:split2].to_numpy(), labels[split1:split2]
test_data_RFE, test_labels_RFE   = df_RFE[split2:], labels[split2:]

## Random Forest

In [102]:
def RandomForest(x_train, x_dev, y_train, y_dev):
    rf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=1)
    rf.fit(x_train, y_train)
    model_pred = rf.predict(x_dev)
    rf_acc = metrics.f1_score(y_dev, model_pred, average="weighted")
    print("The f1 score for RF is {}".format(rf_acc))

In [115]:
# Without feature selection
RandomForest(train_data, dev_data, train_labels, dev_labels)

# With RF feature selection
RandomForest(train_data_RF, dev_data_RF, train_labels_RF, dev_labels_RF)

# With RFE feature selection
RandomForest(train_data_RFE, dev_data_RFE, train_labels_RFE, dev_labels_RFE)

The f1 score for RF is 0.8093219846646126
The f1 score for RF is 0.7795404916829198
The f1 score for RF is 0.8163944239059411


## KNN

In [118]:
# k-NN

# search for an optimal value of K for KNN
k_range = [1,4,7,10]

# list of scores from k_range
f1_scores = []

def KNN(k):
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(train_data, train_labels)
    model_pred = model.predict(dev_data)
    return model_pred

for k in k_range:
    score = metrics.f1_score(dev_labels, KNN(k), average="weighted")
    f1_scores.append(score)
    print("The f1 score for {}-NN is {}".format(k, score))


The f1 score for 1-NN is 0.7557477916049917
The f1 score for 4-NN is 0.7352886099252398
The f1 score for 7-NN is 0.7181820525124354
The f1 score for 10-NN is 0.7111317517786139


## Naive Bayes

ideas from:  
https://stackoverflow.com/questions/14254203/mixing-categorial-and-continuous-data-in-naive-bayes-classifier-using-scikit-lea

https://stackoverflow.com/questions/14274771/choosing-classification-algorithm-to-classify-mix-of-nominal-and-numeric-data?rq=1

https://stackoverflow.com/questions/32707914/different-types-of-features-to-train-naive-bayes-in-python-pandas

handling discrete + continous:  
https://www.quora.com/Machine-Learning/What-are-good-ways-to-handle-discrete-and-continuous-inputs-together/answer/Arun-Iyer-1

In [98]:
# Evaluate binary features and continuous features indepedently (Naive assumption means no correlation between features taken into account anyways)

# BernoulliNB (Binary features only)
train_data_binary = train_data[:,:-10]
dev_data_binary = dev_data[:,:-10]

bnb = BernoulliNB(alpha=1)
bnb.fit(train_data_binary, train_labels)
model_pred = bnb.predict(dev_data_binary)
acc_bnb = metrics.f1_score(dev_labels, model_pred, average="weighted")
print("BernoulliNB model accuracy using binary features only:", acc_bnb)

bnb.predict(dev_data_binary)

BernoulliNB model accuracy using binary features only: 0.48760947746407673


array([1., 5., 7., ..., 5., 5., 5.])

In [99]:
# GaussianNB (Continuous features only)
train_data_cont = train_data[:,-10:]
dev_data_cont= dev_data[:,-10:]

gnb = GaussianNB() 
gnb.fit(train_data_cont, train_labels)
model_pred = gnb.predict(dev_data_cont)
acc_gnb = metrics.f1_score(dev_labels, model_pred, average="weighted")
print("GaussianNB model accuracy using continous features only:", acc_gnb)

gnb.predict(dev_data_cont)

GaussianNB model accuracy using continous features only: 0.5300117538632408


array([7., 2., 7., ..., 1., 5., 5.])

In [73]:
print(bnb.predict_proba(dev_data_binary)[0])
print(gnb.predict_proba(dev_data_cont)[0])

gnb.predict_proba(dev_data_cont)[0] * bnb.predict_proba(dev_data_binary)[0]

[2.24414207e-01 2.16053953e-01 3.86938692e-02 8.85156042e-05
 2.28525380e-01 6.36986956e-02 2.28525380e-01]
[2.40323569e-01 4.36530101e-02 9.85504611e-12 1.55703701e-37
 1.08666966e-09 4.10455489e-10 7.16023419e-01]


array([5.39320232e-02, 9.43140540e-03, 3.81329865e-13, 1.37822072e-41,
       2.48331596e-10, 2.61454793e-11, 1.63629524e-01])

**Improvement ideas:**

Preprocessing:
1. Rescaling bounded features and standardizing continuous features
2. Feature selection

Modeling:
1. Create new features vector from the class assignment probabilities {np.hstack((bernoulli and gaussian))}. Then run another GaussianNB fit on the new features
2. Create function to calculate merged feature probabilities (product of Bernoulli and Gaussian) since we calculate the Bayes probability independently on a specific features

In [22]:
def entropy(target_col):
    """
    Calculate the entropy of a dataset.
    The only parameter of this function is the target_col parameter which specifies the target column
    """
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy


################### 
    
###################


def InfoGain(data,split_attribute_name,target_name="class"):
    """
    Calculate the information gain of a dataset. This function takes three parameters:
    1. data = The dataset for whose feature the IG should be calculated
    2. split_attribute_name = the name of the feature for which the information gain should be calculated
    3. target_name = the name of the target feature. The default for this example is "class"
    """    
    #Calculate the entropy of the total dataset
    total_entropy = entropy(data[target_name])
    
    ##Calculate the entropy of the dataset
    
    #Calculate the values and the corresponding counts for the split attribute 
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    
    #Calculate the weighted entropy
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    
    #Calculate the information gain
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain